# 103590450 馬茂源 四資四

In [1]:
import pandas as pd
import numpy as np
import time
import os
import sys
import string
import html
import hashlib
import re
from collections import defaultdict
import itertools
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
t0 = time.time()

In [3]:
if not os.path.exists('result'):
    os.mkdir('result')

In [4]:
file_names = ['./data/reut2-{0:0>3}.sgm'.format(i) for i in range(22)]
# file_names = [file_names[0]]

## (1) Given the Reuters-21578 dataset, please calculate all kshingles and output the set representation of the text dataset as a matrix. 

In [5]:
def parser(file_name):
    with open(file_name, 'r', encoding='ISO-8859-1') as f:
        file = f.read()
    news = []
    start = 0
    for i in range(len(file)):
        if file[i:i+6] == '<BODY>':
            start = i+6
        elif file[i:i+7] == '</BODY>':
            n = file[start:i].replace('\n', ' ')
            n = n.replace('REUTER &#3;', '')
            news.append(n)
    return news

In [6]:
news = []
for i in file_names:
    each_news = parser(i) 
    news.extend(each_news)

In [7]:
len(news)

19043

In [8]:
news_data = pd.DataFrame(data=news, columns=['news'])

In [9]:
news_data.head()

,news
0,Showers continued throughout the week in the B...
1,Standard Oil Co and BP North America Inc said ...
2,Texas Commerce Bancshares Inc's Texas Commerce...
3,BankAmerica Corp is not under pressure to act ...
4,The U.S. Agriculture Department reported the f...


In [10]:
def tokenizer(text):
    strip_chars = '.' + ' –…' + string.punctuation
    text = html.unescape(text)
    text = text.lower()
    text = text.strip(strip_chars)
    text = text.replace('reuter', '')
    text = re.sub(re.compile('<.*?>'), '', text)
    return re.findall(r'\w+', text)

In [11]:
test_news = news_data['news'][1]

In [12]:
#tokenizer(test_news)

In [13]:
news_data['news_token'] = news_data.apply(lambda x: tokenizer(x['news']), axis=1)

In [14]:
def k_shingle(text, k):
    string = ' '.join(text)
    shingles = set([])
    for i in range(len(string)-k + 1):
        shingles.add(string[i:i+k])
    return (shingles)

In [15]:
news_token = news_data['news_token'].apply(lambda x: k_shingle(x, 5)).values.reshape(-1, 1)
news_token.shape

(19043, 1)

In [16]:
shingles = set([])
for s in news_token:
    shingles |= s[0]
shingles = list(shingles)
len(shingles)

273798

In [17]:
shingles_dict_ = {s:i for i, s in enumerate(shingles)}

In [18]:
def encode_shingles(row, shingles_dict_):
    v = np.zeros(len(shingles_dict_), dtype='int')
    row = row[0]
    v[[shingles_dict_[r] for r in row]] = 1
    return v

In [19]:
shingles = np.apply_along_axis(encode_shingles, 1, news_token, shingles_dict_=shingles_dict_)
shingles.shape

(19043, 273798)

In [20]:
# with open('result/task_1.csv', 'w') as f:
#     for r in shingles:
#         f.write(','.join([i for i in r.astype('str')])+'\n')

In [21]:
np.savetxt("result/task_1.csv", shingles.T[:100, :100], delimiter=',', fmt='%d')

## (2) Given the set representation, compute the minhash signatures of all documents using MapReduce.

In [22]:
news_shingles = shingles.T

In [23]:
def get_prime(greater_than):   
    def is_prime(n):
        if n % 2 == 0 and n > 2: 
            return False
        return all(n % i for i in range(3, int(np.sqrt(n)) + 1, 2))
    is_p = False
    
    while not is_p:
        greater_than += 1
        is_p = is_prime(greater_than)
        
    return greater_than

In [24]:
def get_hash_func_list(n, k=100):
    p = get_prime(n)
    func_list = []
    for a, b in zip(np.random.randint(0, n, size=k),
                   np.random.randint(0, n, size=k)):
        func_list.append(lambda x, a=a,b=b,p=p,n=n: ((a*x+b)%p)%n)
    return np.array(func_list)

In [25]:
def one_pass_minhashing(shingles, k=100):
    n = shingles.shape[0]
    hash_list = get_hash_func_list(n, k=k)
    singnature = np.full((k, shingles.shape[1]), fill_value=np.inf)
    
    for i in range(n):
        hash_value = np.array([h(i) for h in hash_list])

        for j, c in enumerate(shingles[i, :] == 1):
            if c:
                mask = singnature[:, j] > hash_value
                singnature[:, j][mask] = hash_value[mask]
    
    return singnature.astype('int')

In [26]:
test_input = np.array([[1,0,1,0],
                       [1,0,0,1],
                       [0,1,0,1],
                       [0,1,0,1],
                       [0,1,0,1],
                       [1,0,1,0],
                       [1,0,1,0]])
test_singnature = one_pass_minhashing(test_input, k=6)
test_singnature.astype('int')

array([[4, 4, 4, 4],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 2, 0],
       [0, 0, 2, 0],
       [0, 0, 0, 0]])

In [27]:
singnature = one_pass_minhashing(news_shingles, k=100)

c:\python36\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in long_scalars
  


In [28]:
singnature.shape

(100, 19043)

In [29]:
singnature.dtype

dtype('int32')

In [30]:
np.savetxt("result/task_2.csv", singnature, delimiter=",",  fmt='%d')

## (3) Implement the LSH algorithm by MapReduce and output the resulting candidate pairs of similar documents.

In [31]:
hashlib.sha1()

<sha1 HASH object @ 0x000002621260AEE0>

In [32]:
def LSH(singnature, b=20):
    buckets = [defaultdict(set) for i in range(b)]
    k = singnature.shape[0]
    r = k // b
    for i, doc in enumerate(singnature.T):
        x = np.array2string(doc.astype('int'), separator='', precision=0)
        for j, start_idx in enumerate(range(0, k, r)):
            #print(j, j+r)
            key = hashlib.sha1(x[start_idx:start_idx+r].encode()).hexdigest()
            buckets[j][key].add(i)
    return buckets

In [33]:
def get_distance(s1, s2):
    intersection = (s1-s2).astype('int')
    return 1 - (intersection.sum() / intersection.shape[0])

In [34]:
def get_candidate(buckets):
    candidates = set([])
    for bucket in buckets:
        for k, items in bucket.items():
            if len(items) > 1 and len(items) < 100:
                pairs = itertools.combinations(items, 2)
                for p in pairs:
                    if p in candidates:
                        continue
                        
                    if get_distance(singnature[:, p[0]], 
                                    singnature[:, p[1]]) >= 0.8: 
                        candidates.add(p)

    return candidates

In [35]:
buckets = LSH(singnature, b=20)

In [36]:
# for bucket in buckets:
#     print(len(bucket))
#     for _, items in bucket.items():
#         if len(items) > 1 and len(items) < 10000:
#             print('\t', len(items), end=' ')
#             print(len(list(itertools.combinations(items, 2))))

In [37]:
candidates = get_candidate(buckets)

In [38]:
candidates = list(candidates)

In [39]:
len(candidates)

1174935

In [42]:
candidates[:10]

[(107, 11120),
 (10978, 14704),
 (4806, 14311),
 (11913, 1870),
 (5322, 18769),
 (7969, 5943),
 (2044, 2237),
 (11551, 16750),
 (12404, 4385),
 (7179, 15734)]

In [45]:
candidates.sort()

In [46]:
with open('result/task_3.csv', 'w') as f:
    for i, g in itertools.groupby(candidates, key=lambda x: x[0]):
        f.write('%5d, %s\n'%(i, str(list(i[1] for i in g))))

In [47]:
#news_data.loc[new_candidates[0][0]]['news']

In [48]:
#news_data.loc[new_candidates[0][1]]['news']

## (4) Implement K-nearest neighbor (KNN) search using LSH and compare its performance with linear search.

In [49]:
singnature.shape

(100, 19043)

In [50]:
class LSH_Knn:
    def __init__(self, singnature, n_hyper=10, k=3):
        # self._candidates = candidates
        self._singnature = singnature.T
        self.n_hyper = n_hyper
        self.hyper_planes = np.random.randn(self.n_hyper, 
                                            self._singnature.shape[1])
        self.regions = self.get_regions(self._singnature)
        self.k = k
        
    def get_regions(self, singnature):
        return (singnature.dot(self.hyper_planes.T) > 0).astype('int')
    
    def get_distance(self, s1, s2):
        intersection = np.logical_and(s1, s2)
        union = np.logical_or(s1, s2)
        return intersection.sum() / float(union.sum())
    
    def _get_nn(self, singnature, candidates_idx):
        s2 = singnature
        temp_candidates_sing = self._singnature[candidates_idx, :]
        #print(temp_candidates_sing.shape)
        dis = np.apply_along_axis(lambda s1, s2: self.get_distance(s1, s2), 
                            1, 
                            temp_candidates_sing,
                            s2=s2)
        #print(dis)
        idx_of_idx = np.argsort(dis)[:self.k]
        # print(candidates_idx[idx_of_idx])
        return candidates_idx[idx_of_idx]
    
    def _predict(self, singnature):
        r = self.get_regions(singnature)
        nn = np.all(r == self.regions, axis=1).astype('int')
        candidates_idx = np.argwhere(nn==1).reshape(-1,)
        return self._get_nn(singnature, candidates_idx)

In [51]:
model = LSH_Knn(singnature)

In [52]:
N = singnature.T.shape[0]//10

In [53]:
task4_output = ''
t1 = time.time()
for i in range(N):
    idx = model._predict(singnature.T[i])
task4_output += ('[LSH Knn] cost {:.3f} second in {} test data.'
                 .format(time.time()-t1, N))
task4_output

'[LSH Knn] cost 20.547 second in 1904 test data.'

In [54]:
class MyKNeighborsClassifier:
    
    def __init__(self, n_neighbors=3, **kwargs):
        self._k = n_neighbors
        self._X = self._y = None
        self.set_params(**kwargs)
            
    def get_params(self, deep=True):
        # suppose this estimator has parameters "alpha" and "recursive"
        return self.__dict__

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def fit(self, X):
        self._X = X.copy()
        # self._y = y.copy()
        
    def get_distance(self, s1, s2):
        intersection = np.logical_and(s1, s2)
        union = np.logical_or(s1, s2)
        return intersection.sum() / float(union.sum())
    
    def _predict(self, x):
        distances = np.apply_along_axis(lambda x1: get_distance(x, x1), 
                                        1, self._X)
        X_candidates = np.argsort(distances)[:self._k]
        # y_candidates = self._y[X_candidates]
        return X_candidates
    
    def predict(self, X):
        return np.apply_along_axis(lambda x: self._predict(x), 1, X)

In [55]:
model = MyKNeighborsClassifier()
model.fit(singnature.T)

In [56]:
t2 = time.time()
for i in range(N):
    idx = model._predict(singnature.T[i])
task4_output += ('\n[Linear search Knn] cost {:.3f} second in {} test data.'
                 .format(time.time()-t2, N))

print(task4_output)

[LSH Knn] cost 20.547 second in 1904 test data.
[Linear search Knn] cost 236.282 second in 1904 test data.


In [57]:
with open('result/task_4.txt', 'w') as f:
    f.write(task4_output)

In [58]:
print('cost:{:.3f} min'.format((time.time()-t0)/60))

cost:11.047 min
